In [1]:
import findspark
import os
import configparser
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from functools import reduce
from pyspark.sql import DataFrame
import pymongo

### Configure spark context

In [2]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("local")
         .setAppName("My app")
         .set("spark.executor.memory", "1g"))
sc = SparkContext(conf = conf)

### create RDD's in two methods sc.parallelize and sc.textFile

In [3]:
newRDD = sc.parallelize([1,2,3,4,5])

In [5]:
newRDD.count()

5

In [6]:
newRDD1 = sc.textFile("dataset/auto-data.csv")

In [7]:
newRDD1.count()

198

### Saving data to the local file

In [11]:
autoDataFile = open("output/auto-data-saved.csv", "w")

In [12]:
autoDataFile.write("\n".join(newRDD1.collect()))

11350

In [13]:
autoDataFile.close()

### Transformations

### MAP

In [14]:
# replacing every , with a tab space
tsvData = newRDD1.map(lambda x: x.replace(",","\t"))

In [18]:
tsvData.take(5)

['MAKE\tFUELTYPE\tASPIRE\tDOORS\tBODY\tDRIVE\tCYLINDERS\tHP\tRPM\tMPG-CITY\tMPG-HWY\tPRICE',
 'subaru\tgas\tstd\ttwo\thatchback\tfwd\tfour\t69\t4900\t31\t36\t5118',
 'chevrolet\tgas\tstd\ttwo\thatchback\tfwd\tthree\t48\t5100\t47\t53\t5151',
 'mazda\tgas\tstd\ttwo\thatchback\tfwd\tfour\t68\t5000\t30\t31\t5195',
 'toyota\tgas\tstd\ttwo\thatchback\tfwd\tfour\t62\t4800\t35\t39\t5348']

### FILTER

In [23]:
# filtering the data which contains only subaru cars
subData = newRDD1.filter(lambda x: "subaru" in x)

In [24]:
subData.count()

12

### flatMap

In [26]:
words = subData.flatMap(lambda line: line.split(","))
words.count()


144

In [29]:
words.take(10)

['subaru', 'gas', 'std', 'two', 'hatchback', 'fwd', 'four', '69', '4900', '31']

In [5]:
# spark = SparkSession \
#     .builder \
#     .appName("Python Spark SQL basic example") \
#     .config("spark.some.config.option", "some-value") \
#     .getOrCreate()

SyntaxError: invalid syntax (<ipython-input-7-939bbb97c795>, line 1)

In [11]:
rdd = spark.range(0,100).toDF()

IllegalArgumentException: "requirement failed: The number of columns doesn't match.\nOld column names (1): id\nNew column names (0): "